In [30]:
from datetime import date
import os
import re
import pandas as pd

# Месяцы в родительном падеже
MONTHS = {
    "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
    "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12,
}
MONTH_RE = r"(января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)"
DASH_RE = r"[—–-]"  # эм-, эн-, дефис

# Паттерны: сначала «день месяц — день месяц», затем «день—день месяц», затем одиночная дата
PATTERN_CROSS_MONTH = re.compile(
    rf"^(\d{{1,2}})\s+{MONTH_RE}\s*{DASH_RE}\s*(\d{{1,2}})\s+{MONTH_RE}\b",
    re.IGNORECASE,
)
PATTERN_SAME_MONTH_RANGE = re.compile(
    rf"^(\d{{1,2}})\s*{DASH_RE}\s*(\d{{1,2}})\s+{MONTH_RE}\b",
    re.IGNORECASE,
)
PATTERN_SINGLE_DATE = re.compile(
    rf"^(\d{{1,2}})\s+{MONTH_RE}\b",
    re.IGNORECASE,
)

def _month_num(name: str) -> int:
    return MONTHS[name.lower()]

def parse_events_file(source_path: str, out_path: str):
    # Год из имени файла (например, "2001.txt" → "2001")
    m_year = re.search(r"(\d{4})", os.path.basename(source_path))
    if m_year:
        year = int(m_year.group(1))
    else:
        print(f"⚠️ Не удалось распознать год из названия файла: {source_path}")
        return

    with open(source_path, "r", encoding="utf-8") as f:
        text = f.read()

    # По строкам, без пустых
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    data = []
    current_start = None
    current_end = None

    for line in lines:
        # 1) Диапазон с разными месяцами: «25 июля—23 сентября …»
        m = PATTERN_CROSS_MONTH.match(line)
        if m:
            d1, m1, d2, m2 = m.groups()
            y_start = year
            y_end = year
            # если переход через год (например, «декабря—января») — увеличим год конца
            if _month_num(m2) < _month_num(m1) and year:
                # конец в следующем календарном году
                y_end = y_start + 1

            current_start = date(y_start, _month_num(m1), int(d1))
            current_end = date(y_end, _month_num(m2), int(d2))
            event = line[m.end():].lstrip(" \t:—–-—").strip()
            if event:
                data.append((current_start, current_end, event))
            continue

        # 2) Диапазон в одном месяце: «1–30 января …»
        m = PATTERN_SAME_MONTH_RANGE.match(line)
        if m:
            d1, d2, m1 = m.groups()
            current_start = date(int(year), _month_num(m1), int(d1))
            current_end = date(int(year), _month_num(m1), int(d2))
            event = line[m.end():].lstrip(" \t:—–-—").strip()
            if event:
                data.append((current_start, current_end, event))
            continue

        # 3) Одиночная дата: «2 января …» или «2 января:» (без события на той же строке)
        m = PATTERN_SINGLE_DATE.match(line)
        if m:
            d1, m1 = m.groups()
            current_start = date(int(year), _month_num(m1), int(d1))
            current_end = None
            event = line[m.end():].lstrip(" \t:—–-—").strip()
            if event:
                data.append((current_start, current_end or "", event.capitalize()))
            continue

        # 4) Строка-событие под текущей датой/диапазоном
        if current_start:
            data.append((current_start, current_end or "", line))
        else:
            print(f"⚠️ Не удалось распарсить строку: {line}")

    df = pd.DataFrame(data, columns=["date_start", "date_end", "event"])

    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"✅ [{year or '—'}] Экспортировано {len(df)} строк → {out_path}")

def parse_raw_events(folder_path: str, out_path: str):
    """
    Обходит все .txt в папке и парсит каждый.
    """
    files = sorted(f for f in os.listdir(folder_path) if f.lower().endswith(".txt"))

    for name in files:
        source_path = os.path.join(folder_path, name)
        csv_name = os.path.splitext(name)[0] + ".csv"
        csv_path = os.path.join(out_path, csv_name)
        parse_events_file(source_path, csv_path)


parse_raw_events("events/1_raw", 'events/2_struct')

✅ [2000] Экспортировано 307 строк → events/2_struct\2000.csv
✅ [2001] Экспортировано 239 строк → events/2_struct\2001.csv
✅ [2002] Экспортировано 482 строк → events/2_struct\2002.csv
✅ [2003] Экспортировано 188 строк → events/2_struct\2003.csv
✅ [2004] Экспортировано 136 строк → events/2_struct\2004.csv
✅ [2005] Экспортировано 183 строк → events/2_struct\2005.csv
✅ [2006] Экспортировано 110 строк → events/2_struct\2006.csv
✅ [2007] Экспортировано 128 строк → events/2_struct\2007.csv
✅ [2008] Экспортировано 146 строк → events/2_struct\2008.csv
✅ [2009] Экспортировано 147 строк → events/2_struct\2009.csv
✅ [2010] Экспортировано 170 строк → events/2_struct\2010.csv
✅ [2011] Экспортировано 289 строк → events/2_struct\2011.csv
✅ [2012] Экспортировано 241 строк → events/2_struct\2012.csv
✅ [2013] Экспортировано 247 строк → events/2_struct\2013.csv
✅ [2014] Экспортировано 287 строк → events/2_struct\2014.csv
✅ [2015] Экспортировано 247 строк → events/2_struct\2015.csv
✅ [2016] Экспортировано 